# OFDS Data Visualisation Notebook

This notebook loads OFDS data and visualises it on a map.

Your data files must each be a JSON-formatted [OFDS network package](https://open-fibre-data-standard.readthedocs.io/en/latest/reference/publication_formats/json.html#small-files-and-api-responses-option). You can use the [OFDS Convert, Validate, Explore tool](https://ofds.cove.opendataservices.coop/) to convert your data between different publication formats.

## Setup

Install requirements.

In [ ]:
#@title
%pip install ipyleaflet ipywidgets seaborn > pip-log.txt

!rm -rf data
!mkdir data

## Load data

If you are using Google Colaboratory to execute this notebook, you can use the cells in this section to load your data files. Otherwise, you must place the files in the `/data` directory.

### From your local file system

In [ ]:
#@title
from google.colab import files

uploaded = files.upload()

for filename in uploaded.keys():
  !mv {filename} data/{filename}

### From a Google Drive folder

In [ ]:
#@title
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

folder_id = input('Enter the ID of the Google Drive folder containing your data ')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_list = drive.ListFile({
    'q': f"'{folder_id}' in parents and trashed=false",
    'includeItemsFromAllDrives': True,
    'supportsAllDrives': True,
  }).GetList()

for drive_file in file_list:
  drive_file.GetContentFile(f'data/{drive_file["title"]}')


## Render map

In [10]:
#@title
import json
import os
import seaborn as sns

from ipyleaflet import (
  CircleMarker,
  LayerGroup,
  LayersControl,
  Map,
  Polyline
)

from ipywidgets import (
    HTML,
    Layout
)

# Load data
networks = []

data_directory = 'data'

for filename in os.listdir(data_directory):
  path = os.path.join(data_directory, filename)

  if os.path.isfile(path):
    with open(path, 'r') as f:
        data = json.load(f)

    if 'networks' in data:
      for network in data['networks']:
        networks.append(network)
    else:
      print(f"{path} is not an OFDS network package (missing 'networks' key)")

# Generate a palette with a different colour for each network
palette = sns.color_palette(None, len(networks)).as_hex()

# Generate the map
m = Map(
    center = (7.9465, 1.0232),
    zoom = 7,
    layout = Layout(width='80%', height='600px')
    )

# Add each network to the map
layer_groups = []

for network in networks:
    
    group = LayerGroup(
        name = network['name']
    )

    color = palette[networks.index(network)]

    for span in network.get('spans', []):
        
        popup = HTML(
            value = f"""
              Name: {span.get('name', '')}</br>
              Phase: {span['phase'].get('name') if 'phase' in span else ''}</br>
              Status: {span.get('status', '')}</br>
              Ready for service date: {span.get('readyForServiceDate', '')}</br>
              Physical infrastructure provider: {span['physicalInfrastructureProvider'].get('name', '') if 'physicalInfrastructureProvider' in span else ''}</br>
              Network providers: {', '.join([organisationReference['name'] if 'name' in organisationReference else '' for organisationReference in span['networkProviders']]) if 'networkProviders' in span else ''}</br>
              Supplier: {span['physicalInfrastructureProvider'].get('name', '') if 'physicalInfrastructureProvider' in span else ''}</br>
              Transmission medium: {', '.join(span.get('transmissionMedium', []) if 'transmissionMedium' in span else '')}</br>
              Deployment: {', '.join(span.get('deployment', []) if 'deployment' in span else '')}</br>
              Dark fibre availability: {span.get('darkFibre', '')}</br>
              Fibre type: {span.get('fibreType', '')}</br>
              Fibre count: {span.get('fibreCount', '')}</br>
              Fibre length: {span.get('fibreLength', '')}</br>
              Technologies: {', '.join(span.get('technologies', []) if 'technologies' in span else '')}</br>
              Capacity: {span.get('capacity', '')}Gbps

            """
        )

        line = Polyline(
            locations = [[position[1],position[0]] for position in span['route']['coordinates']],
            color = color,
            fill = False
        )

        line.popup = popup

        group.add_layer(line)
    
    for node in network.get('nodes', []):
        
        popup = HTML(
            value = f"""
              Name: {node.get('name', '')}</br>
              Phase: {node['phase'].get('name') if 'phase' in node else ''}</br>
              Status: {node.get('status', '')}</br>
              Type: {', '.join(node.get('type', [])) if 'type' in node else ''}</br>
              Access point: {node.get('accessPoint', '')}</br>
              Power availability: {node.get('power', '')}</br>
              Technologies: {', '.join(node.get('technologies', []) if 'technologies' in node else '')}</br>
              Physical infrastructure provider: {node['physicalInfrastructureProvider'].get('name', '') if 'physicalInfrastructureProvider' in node else ''}</br>
              Network providers: {', '.join([organisationReference['name'] if 'name' in organisationReference else '' for organisationReference in node['networkProviders']]) if 'networkProviders' in node else ''}
            """
        )

        marker = CircleMarker(
            location = (node['location']['coordinates'][1], node['location']['coordinates'][0]),
            radius = 2,
            color = color,
            fill_color = color
        )

        marker.popup = popup

        group.add_layer(marker)
    
    layer_groups.append(group)

for group in layer_groups:
    m.add_layer(group)

m.add_control(LayersControl(position='topright'))

m

Map(center=[7.9465, 1.0232], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom…